In [1]:
DATASET_PATH = 'Dataset/'

In [3]:
import pandas as pd
from scipy.io import wavfile
import os
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

sns.set(style="whitegrid", color_codes=True)
%matplotlib inline

# set seed for reproducibility
np.random.seed(0)

In [12]:
# print the content of 
contents = os.listdir(DATASET_PATH)
for c in contents:
    print(c)

cv-invalid
cv-invalid.csv
cv-other-dev
cv-other-dev.csv
cv-other-test
cv-other-test.csv
cv-other-train
cv-other-train.csv
cv-valid-dev
cv-valid-dev.csv
cv-valid-test
cv-valid-test.csv
cv-valid-train
cv-valid-train.csv
LICENSE.txt
README.txt


In [16]:
# import datasets
df_train = pd.read_csv(DATASET_PATH + 'cv-valid-train.csv')
df_test = pd.read_csv(DATASET_PATH + 'cv-valid-test.csv')

In [17]:
print(f'Train set shape: {df_train.shape}')
print(f'Test set shape: {df_test.shape}')

Train set shape: (195776, 8)
Test set shape: (3995, 8)
